In [1]:
from pyspark import SparkContext
import sys
import csv
from collections import Counter
from itertools import combinations

input_file = "../data/small1.csv"
support = 4

In [2]:
def iterm_num(iterator):
    cnt = 0
    for x in iterator:
        cnt += 1
    yield cnt

In [19]:
def A_priori(baskets, support, total_baskets_cnt):
    clean_baskets = list(baskets)
    local_support = support * len(clean_baskets) / float(total_baskets_cnt)

    max_itemset_size = max([len(basket) for basket in clean_baskets])
    # candidate_itemsets = {str(i): [] for i in range(1, max_itemset_size + 1)}
    candidate_itemsets = []

    # frequent items
    single_cnt = Counter()
    for basket in clean_baskets:
        single_cnt.update(basket)

    candidate_single = sorted([i for i in single_cnt if single_cnt[i] >= local_support])
    former_candidate = candidate_single
    # candidate_itemsets['1'].extend(former_candidate)
    candidate_itemsets.extend(former_candidate)

    # frequent itemsets with size >= 2
    for itemset_size in range(2, max_itemset_size + 1):
        tmp_itemsets = []
        for basket in clean_baskets:
            if len(basket) < itemset_size:
                continue
            for itemset in combinations(basket, itemset_size):
                itemset = sorted(itemset)
                if itemset_size == 2:
                    immediate_subset = [i[0] for i in combinations(itemset, itemset_size - 1)]
                else:
                    immediate_subset = [sorted(i) for i in combinations(itemset, itemset_size - 1)]
                immediate_subset_is_candidate = [i in former_candidate for i in immediate_subset]
                if all(immediate_subset_is_candidate):
                    tmp_itemsets.append('_'.join(itemset))
        itemset_cnt = Counter(tmp_itemsets)
#         former_candidate = [i.split('_') for i in itemset_cnt if itemset_cnt[i] >= local_support]
        former_candidate = [i for i in itemset_cnt if itemset_cnt[i] >= local_support]
        # candidate_itemsets[str(itemset_size)].extend(former_candidate)
        candidate_itemsets.extend(former_candidate)

    return candidate_itemsets


In [37]:
def candidate_count(baskets,candidate_itemsets):
    itemsets_count = []
    baskets = list(baskets)
    for basket in baskets:
        for itemset in candidate_itemsets:
            if all([item in basket for item in itemset.split('_')]):
                itemsets_count.append(itemset)
    return Counter(itemsets_count).items()

In [35]:
basket = ['101','100','99','98']
itemset1 = '101_99_98'
itemset2 = '101_102'
print(all([item in basket for item in itemset1.split('_')]))
print(all([item in basket for item in itemset2.split('_')]))

True
False


In [4]:
input_file = "../data/small1.csv"

sc = SparkContext.getOrCreate()
raw_data = sc.textFile(input_file)
header = raw_data.first()
raw_data_without_header = raw_data.filter(lambda x: x != header)
clean_data = raw_data_without_header.map(lambda line: [i for i in line.strip().split(',')])

baskets = clean_data.map(lambda x: (x[0], [x[1]])).reduceByKey(lambda a, b: a + b).map(
        lambda x: [x[0], sorted(list(set(list(x[1]))))])
# baskets2 = clean_data.map(lambda x: (x[1], [x[0]])).reduceByKey(lambda a, b: a + b).map(
#         lambda x: [x[0], sorted(list(set(list(x[1]))))])

# get rid of original key:
#   case 1 only remain [business_ids];
#   case 2 only remain [user_ids]
clean_baskets = baskets.map(lambda x: x[1])
clean_baskets.persist()
total_baskets_cnt = clean_baskets.count()
print(clean_baskets.collect())

[['100', '101', '102', '98'], ['101', '102', '103', '97', '99'], ['102', '103', '104', '97', '98', '99'], ['97', '98', '99'], ['97', '98'], ['100', '101', '102', '105', '106', '107', '108', '98'], ['97'], ['100', '101', '98', '99'], ['97', '99'], ['102', '97', '98'], ['100', '101', '97', '99'], ['102', '103', '105', '97', '98', '99'], ['97', '98'], ['101', '102'], ['101', '97', '99'], ['97', '98', '99'], ['100', '101', '102', '103', '105', '106', '107', '108', '98', '99'], ['101', '97', '99'], ['97', '98', '99']]


In [20]:
phase1_map = clean_baskets.mapPartitions(lambda baskets_subset: A_priori(baskets_subset, support, total_baskets_cnt)).map(lambda x: (x,1))

In [21]:
phase1_map.collect()

[('100', 1),
 ('101', 1),
 ('102', 1),
 ('97', 1),
 ('98', 1),
 ('99', 1),
 ('100_101', 1),
 ('100_98', 1),
 ('101_102', 1),
 ('101_98', 1),
 ('102_98', 1),
 ('102_97', 1),
 ('97_99', 1),
 ('97_98', 1),
 ('98_99', 1),
 ('100', 1),
 ('101', 1),
 ('102', 1),
 ('103', 1),
 ('105', 1),
 ('97', 1),
 ('98', 1),
 ('99', 1),
 ('100_101', 1),
 ('100_99', 1),
 ('101_97', 1),
 ('101_99', 1),
 ('97_99', 1),
 ('102_103', 1),
 ('102_105', 1),
 ('102_98', 1),
 ('102_99', 1),
 ('103_105', 1),
 ('103_98', 1),
 ('103_99', 1),
 ('105_98', 1),
 ('105_99', 1),
 ('97_98', 1),
 ('98_99', 1),
 ('101_102', 1)]

In [32]:
phase1_reduce = phase1_map.reduceByKey(lambda x,y: 1).keys().collect()

In [33]:
phase1_reduce

['100',
 '102',
 '98',
 '101_98',
 '102_97',
 '97_99',
 '102_103',
 '102_105',
 '103_105',
 '103_98',
 '105_98',
 '101',
 '97',
 '99',
 '100_101',
 '100_98',
 '101_102',
 '102_98',
 '97_98',
 '98_99',
 '103',
 '105',
 '100_99',
 '101_97',
 '101_99',
 '102_99',
 '103_99',
 '105_99']

In [49]:
phase2_map = clean_baskets.mapPartitions(lambda baskets_subset: candidate_count(baskets_subset,phase1_reduce))
phase2_reduce = phase2_map.reduceByKey(lambda x,y: x+y).filter(lambda kv: kv[1]>=support).keys().map(lambda x: x.split('_')).collect()

In [50]:
phase2_reduce

[['100'],
 ['102'],
 ['98'],
 ['101', '98'],
 ['102', '97'],
 ['97', '99'],
 ['102', '103'],
 ['101'],
 ['100', '101'],
 ['100', '98'],
 ['101', '102'],
 ['102', '98'],
 ['97'],
 ['99'],
 ['103'],
 ['101', '97'],
 ['101', '99'],
 ['102', '99'],
 ['103', '99'],
 ['97', '98'],
 ['98', '99']]

In [16]:
item_cnt = clean_data.map(lambda x: [x[1],1]).reduceByKey(lambda a,b: a + b)
custum_cnt = clean_data.map(lambda x: [x[0],1]).reduceByKey(lambda a,b: a + b)

In [19]:
frequent_item = item_cnt.filter(lambda x: x[1] > support)
frequent_item.collect()

[('100', 5), ('98', 12), ('102', 8), ('101', 9), ('99', 12), ('97', 14)]

In [21]:
frequent_custum = custum_cnt.filter(lambda x: x[1] > support)
frequent_custum.collect()

[('4', 5), ('8', 6), ('12', 8), ('3', 6), ('13', 10)]